In [ ]:
from nbtemplate import display_header, get_path, display_codetoggle
display_header('Plots.ipynb')

## Goal

This notebook shows a few plots to give a flavor of how the data taken with the Lynx XGS might look.

In [ ]:
import numpy as np
from astropy.table import Table
from astropy.coordinates import SkyCoord
from marxs.source import PointSource, FixedPointing
from marxslynx import lynx

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
instrum = lynx.PerfectLynx()

In [ ]:
EQPegAspec = Table.read('../inputdata/EQPegA_flux.tbl', format='ascii', names=['energy', 'flux'])
# Restrict to the range where the Lynx XGS tables are defined
EQPegAspec = EQPegAspec[(EQPegAspec['energy'] > 0.2) & (EQPegAspec['energy'] < 2.)]

In [ ]:
coord = SkyCoord.from_name("EQ Peg A")
eqpega = PointSource(coords=coord, energy=EQPegAspec,
                    geomarea=instrum.elements[0].area,
                    flux=(EQPegAspec['flux'][1:] * np.diff(EQPegAspec['energy'])).sum())
eqpegapointing = FixedPointing(coords=coord)

In [ ]:
photons = eqpega.generate_photons(1e3)
len(photons)

In [ ]:
photons = eqpegapointing(photons)
photons = instrum(photons)

In [ ]:
from astropy.visualization import (MinMaxInterval, AsymmetricPercentileInterval, LogStretch, SqrtStretch,
                                   ImageNormalize)

bins = [np.linspace(400, 700, 500), np.linspace(-20, 20, 500)]

H, xe, ye = np.histogram2d(photons['projcirc_y'], photons['projcirc_z'], weights=photons['probability'],
               bins=bins)
# Create an ImageNormalize object
norm = ImageNormalize(H, interval=MinMaxInterval(), stretch=LogStretch())

with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(figsize=(10, 6))
    im = ax.imshow(H.T, norm=norm, origin='lower', extent=(xe[0], xe[-1], ye[0], ye[-1]),
                       aspect='auto', cmap=plt.get_cmap('magma'))
    cbar1=plt.colorbar(im, ax=ax, label='counts/bin') #, ticks=[0, 10, 30, 100, 300])
    ax.grid(False)
    fig.suptitle('Diffraction by L1 support bars', fontsize=40)   
    ax.set_xlabel('Dispersion direction in focal plane (mm)')
    ax.set_ylabel('Cross-dispersion direction (mm)')

fig.savefig(get_path('figures') + '/EQPegdetim.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/EQPegdetim.pdf', bbox_inches='tight')


Simulated detector image of an emission line source (the model for the simulation matches the active star EQ Peg A). Note the x and y axis are shown in very different scales. Counts are shown on a logarithmic scale. The bright band of varying thickness in the middle is the main signal, above and below are rays that are dispersed by the L1 support bars. The simulation includes neither astrophysical nor instrumental background.

In the center of the image is the main signal. This distribution is wider on the left and right on the image and goes through a minimum around 600~mm. The Rowland torus optimizes the spectral resolution, i.e.\ the width of the signal in dispersion direction, but at the cost of a wider distribution in cross-dispersion direction. In the tilted torus layout we have chosen for the XGS, there are two minima in cross-dispersion direction. The first one is at the position of the zeroth order (not shown here), the second one is located where the Rowland circle intersects the symmetry axis of the torus again.

Above and below the main stripe of signal is a large area with photons that looks almost like a diffuse emission region. This is signal dispersed by the L1 structures.

Above and below the main stripe of signal is a large area with photons that looks almost like a diffuse emission region. This is signal dispersed by the L1 structures. The dispersion angle is given by the diffraction equation that governs any diffractive grating:
$ n \lambda = d \sin(\alpha) $
where $d$ is the period of the structure, $\alpha$ is the dispersion angle,
$\lambda$ is the wavelength of the photons in question and $n$ the diffraction
order. Almost 90\% of the signal is found in the $n=0$ order (the central
strip), but a few \% go into orders $n_{\mathrm{L1}} = -3$, -2, -1, 1, 2, and 3. For a CAT grating, at any particular position in the dispersion direction,
there are photons of different energies (see Figure below), at
some positions photons of eight to ten different energies fall onto the same
detector location. That means that ten different $\lambda$ values are possible
in the equation above, and each of them can be seen in $n_{\mathrm{L1}}=-3$ to 3. Altogether
this means that the L1 dispersed orders are not separately visible in the
image, but smear together although in some cases, bright emission lines can be seen
individually. For example, around dispersion coordinate 550~mm, there is a single bright
emission line, that can be seen individually above and below the main signal,
too.


In [ ]:

bins = [np.linspace(400, 700, 500), np.linspace(0, 1.5, 200)]

H, xe, ye = np.histogram2d(photons['projcirc_y'], 
                           photons['energy'] + np.random.normal(scale=.060/2.355, size=len(photons)),
                           weights=photons['probability'],
               bins=bins)
# Create an ImageNormalize object
norm = ImageNormalize(H, interval=MinMaxInterval(), stretch=LogStretch())

with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(figsize=(10, 6))
    im = ax.imshow(H.T, norm=norm, origin='lower', extent=(xe[0], xe[-1], ye[0], ye[-1]),
                       aspect='auto', cmap=plt.get_cmap('magma'))
    cbar1=plt.colorbar(im, ax=ax, label='counts/bin') #, ticks=[0, 10, 30, 100, 300])
    ax.grid(False)
    ax.set_xlabel('Dispersion direction in focal plane (mm)')
    ax.set_ylabel('Energy (keV)')
    fig.suptitle('Detector has 60 eV FWHM', fontsize=40)
    
fig.savefig(get_path('figures') + '/EQPegensort.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/EQPegensort.pdf', bbox_inches='tight')

Simulation of EQ Peg A spectrum shown above, but integrated in cross-dispersion direction. The
different orders in this energy-position plot can be discerned by eye. the
color scale is logarithmic to highlight faint features. While orders are
generally well separated, bright emission lines have enough signal that they
can contribute a non-negligible number of photons to neighboring orders.

In [ ]:
fig, axes = plt.subplots(4, 5, sharex=True, sharey=True, subplot_kw={'aspect': 'equal'}, figsize=(12, 7))

for i in range(18):
    ind = photons['CCD_ID'] == i
    hist, xedges, yedges = np.histogram2d(photons['detpix_x'][ind], photons['detpix_y'][ind], 
                                          weights=photons['probability'][ind],
                            bins=[np.arange(0, 1024, 8), np.arange(0, 1024, 8)])

    norm = ImageNormalize(hist, interval=AsymmetricPercentileInterval(0, 99.9),
                          stretch=SqrtStretch())

    ax = axes[np.unravel_index(i, axes.shape)]
    cax = ax.imshow(hist.T, extent=[0, 1024, 0, 1024],
                origin='lower', aspect='equal', 
                cmap=plt.get_cmap('inferno'),
                norm=norm)
    ax.set_xlim(0, 1024)
    ax.set_ylim(0, 1024)
    ax.set_title(i+1)

Figure showing images for the 18 individual detectors planned for the XGS for the same simulations as above.

In [ ]:
display_codetoggle()